# Kernel Tricks

Demonstrating different SVM kernels for non-linear classification.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.datasets import make_moons, make_circles

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 10)
np.random.seed(42)

---
## Compare Kernels on Non-Linear Data

In [ ]:
# Generate non-linear data
X_moons, y_moons = make_moons(n_samples=200, noise=0.2, random_state=42)
X_circles, y_circles = make_circles(n_samples=200, noise=0.1, factor=0.5, random_state=42)

datasets = [(X_moons, y_moons, 'Moons'), (X_circles, y_circles, 'Circles')]
kernels = ['linear', 'poly', 'rbf']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for row, (X, y, name) in enumerate(datasets):
    for col, kernel in enumerate(kernels):
        # Train SVM
if kernel == 'poly':
            svm = SVC(kernel=kernel, degree=3, C=1, gamma='auto')
        else:
            svm = SVC(kernel=kernel, C=1, gamma='auto')
        
        svm.fit(X, y)
        
        # Create mesh
        x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
        y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                             np.linspace(y_min, y_max, 200))
        
        # Predict
        Z = svm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        # Plot
        axes[row, col].contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
        axes[row, col].scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu',
                              edgecolors='k', s=30)
        
        acc = svm.score(X, y)
        axes[row, col].set_title(f'{name} - {kernel.upper()}\nAcc: {acc*100:.1f}%',
                                fontsize=12, fontweight='bold')
        axes[row, col].set_xlabel('Feature 1')
        axes[row, col].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

### Observations:
- **Linear**: Fails on non-linear data
- **Polynomial**: Works for specific curvature
- **RBF**: Flexible, works well on both datasets

---
## Effect of Gamma (RBF Kernel)

In [ ]:
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)
gammas = [0.1, 1, 10]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, gamma in enumerate(gammas):
    svm = SVC(kernel='rbf', gamma=gamma, C=1)
    svm.fit(X, y)
    
    # Mesh
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    
    axes[idx].contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    axes[idx].scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='k', s=30)
    
    acc = svm.score(X, y)
    axes[idx].set_title(f'γ={gamma}, Acc={acc*100:.1f}%',
                       fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Feature 1')
    axes[idx].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

print('Low γ → smooth boundary')
print('High γ → complex boundary (overfitting risk)')

---
## Summary

### Kernel Selection:
- **Linear**: Fast, interpretable, use when data linearly separable
- **RBF**: Good default for non-linear, flexible
- **Polynomial**: Specific polynomial relationships

### Hyperparameters:
- **C**: Margin vs violations trade-off
- **γ**: Kernel width (RBF), affects boundary complexity

### Key Point:
"Kernels allow SVM to create non-linear decision boundaries by implicitly mapping data to higher dimensions. RBF kernel is a good default - tune C and γ via cross-validation."